In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Adesh Mishra\\Documents\\Chicken-Disease-Classification-\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Adesh Mishra\\Documents\\Chicken-Disease-Classification-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Data Ingestion Configuration
    """
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config    

    

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size



In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info:\n {headers}")

        else:
            logger.info(f"File alraedy exists of size:{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip file path:str
        extracts the zip file into directory
        fub=nctioinn return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2025-06-24 12:13:13,412: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-24 12:13:13,423: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-24 12:13:13,428: INFO: common: Created directory: artifacts]
[2025-06-24 12:13:13,436: INFO: common: Created directory: artifacts/data_ingestion]


[2025-06-24 12:13:43,615: INFO: 299252156: artifacts/data_ingestion/data.zip downloaded with following info:
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 92EA:2928A9:3C5FD4:AE3F35:685A4905
Accept-Ranges: bytes
Date: Tue, 24 Jun 2025 06:43:18 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4734-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1750747397.342882,VS0,VE903
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 5f3744948bb7b233a42260034d3316b33994a3cb
Expires: Tue, 24 Jun 2025 06:48:18 GMT
Source-Age: 1

]
